In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
import pandas_profiling as pf

In [ ]:
data = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
print(len(data))
print(len(data.columns))

print(data.dtypes)
labels = data["SalePrice"]
data = data.loc[:, data.columns != "SalePrice"]
feature_names = list(data)
print("--------------------")
object_cols = []
for feature_name in feature_names:
    if data[feature_name].dtypes == object:
        object_cols.append(feature_name)
        number = data[feature_name].nunique()
        unique_names = data[feature_name].unique()
        print(feature_name + ' : ' + str(number) + ' : ' + str(unique_names.tolist()))
# print(object_cols)
data_obj = data[data.columns[data.dtypes == object]]
data_non_obj = data[data.columns[data.dtypes != object]]
print(len(data.columns))
print(len(data_obj.columns))
print(len(data_non_obj.columns))
# print(data_non_obj)
house_profile = pf.ProfileReport(data)
house_profile.to_file(output_file='houseprofile.html')

1460
81
Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
                  ...   
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual       object
TotRmsAbvGrd       int64
Functional        object
Fireplaces         int64
FireplaceQu       object
GarageType        object
GarageYrBlt      

Transform object variables to one hot encoding and remove one column from each variable

In [86]:
feature_names = data_obj.columns
print(len(data_obj.columns))
d = []
new_df = pd.DataFrame()
for feature in feature_names:
    temp = pd.get_dummies(data_obj[feature])
    temp_list = list(temp)
    temp = temp.drop(columns = temp_list[0])
    new_df = pd.concat([new_df, temp], axis=1)

print(len(new_df.columns))

43
209


Normalize other variables that are integer or float

In [88]:
# remove id column
data_id = data_non_obj['Id']
data_numbers = data_non_obj.loc[:, data_non_obj.columns != 'Id']
print(data_numbers)
x = data_numbers.values  # returns a numpy array
# same scaler for test data (transform instead of fit transform)
x_scaled = min_max_scaler.fit_transform(x)
data_numbers_normal = pd.DataFrame(x_scaled)
# data_numbers_normal = pd.concat([data_id, data_numbers_normal], axis=1)
print(data_numbers_normal)

      MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0             60         65.0     8450            7            5       2003   
1             20         80.0     9600            6            8       1976   
2             60         68.0    11250            7            5       2001   
3             70         60.0     9550            7            5       1915   
4             60         84.0    14260            8            5       2000   
5             50         85.0    14115            5            5       1993   
6             20         75.0    10084            8            5       2004   
7             60          NaN    10382            7            6       1973   
8             50         51.0     6120            7            5       1931   
9            190         50.0     7420            5            6       1939   
10            20         70.0    11200            5            5       1965   
11            60         85.0    11924            9 

put all data together

In [99]:
data_processed = pd.concat([data_numbers_normal,new_df], axis=1)
# # df= data_processed.fillna(data_processed.mean())
# df = data_processed.fillna(value = data_processed.mean())
# dframe["Age"].replace(np.nan,age_array.mean())
df = data_processed
mean_train = data_processed.mean(axis = 0)
print(len(mean_train))
print(len(df.columns))
index = 0
for feature in list(data_processed):
#     df[feature] = data_processed[feature].replace(np.nan,data_processed[feature].mean())
    print(mean_train[index])
    df[feature] = data_processed[feature].replace(np.nan,mean_train[index])
    index = index + 1
#     mean_train.append(data_processed[feature].mean())
    
# print(df)
print(df.isnull().values.any())

227
227
0.21704270749395832
0.16797930947954287
0.04308036215939503
0.5665905631659062
0.571917808219178
0.7193319436172311
0.5810958904109572
0.06480328856749315
0.07860377853072238
0.0315802680247579
0.24282551838994165
0.1730653767683787
0.19013921806534403
0.16803509237453965
0.01021769326563847
0.22258170659949852
0.14178082191780916
0.028767123287671233
0.5216894977168943
0.19143835616438357
0.3583047945205479
0.34885844748857775
0.3764840182648397
0.20433789954337941
0.7136923989715875
0.4417808219178082
0.3335543984388589
0.10997026901743896
0.08530214620219882
0.03977193766130632
0.0067117894509761645
0.03137699771689497
0.0037383524520176706
0.00280574458683164
0.48381070983810615
0.45393835616438355


KeyError: 36

In [90]:
print(len(mean_train))
print(len(df.columns))
print(type(mean_train[:]))

245
245
<class 'list'>


Save preprocessed data to file

In [91]:
df.to_csv(r'C:\Users\Sina\Desktop\Anaconda\Housing-Price\preprocessed_data.csv' ,index=False)
labels.to_csv(r'C:\Users\Sina\Desktop\Anaconda\Housing-Price\train_labels.csv' ,index=False, header = True)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Sina\\Desktop\\Anaconda\\Housing-Price\\preprocessed_data.csv'

prepare test data

In [94]:
test_data = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')
data_obj_test = test_data[test_data.columns[test_data.dtypes == object]]
data_non_obj_test = test_data[test_data.columns[test_data.dtypes != object]]
print(len(data_obj_test.columns))
print(len(data_non_obj_test.columns))
feature_names = data_obj_test.columns
new_df = pd.DataFrame()
for feature in feature_names:
    temp = pd.get_dummies(data_obj_test[feature])
    temp_list = list(temp)
    temp = temp.drop(columns = temp_list[0])
    new_df = pd.concat([new_df, temp], axis=1)
print(len(new_df.columns))
data_id = data_non_obj['Id']
data_numbers = data_non_obj.loc[:, data_non_obj.columns != 'Id']
x = data_numbers.values  # returns a numpy array
# same scaler for test data (transform instead of fit transform)
x_scaled = min_max_scaler.transform(x)
data_numbers_normal = pd.DataFrame(x_scaled)
data_processed = pd.concat([data_numbers_normal,new_df], axis=1)
index = 0
df = data_processed
for feature in list(data_processed):
    mean_num = mean_train[index]
    print(mean_num)
    df[feature] = data_processed[feature].replace(np.nan,mean_num)
    index = index + 1
print(df.isnull().values.any())
df.to_csv(r'C:\Users\Sina\Desktop\Anaconda\Housing-Price\test_ready.csv' ,index=False, header = True)

43
37
191
0.21704270749395832
0.16797930947954226
0.04308036215939503
0.5665905631659062
0.571917808219178
0.7193319436172311
0.5810958904109572
0.06480328856749315
0.07860377853072238
0.0315802680247579
0.24282551838994165
0.1730653767683787
0.19013921806534403
0.16803509237453965
0.01021769326563847
0.22258170659949852
0.14178082191780916
0.028767123287671233
0.5216894977168943
0.19143835616438357
0.3583047945205479
0.34885844748857775
0.3764840182648397
0.20433789954337941
0.7136923989715884
0.4417808219178082
0.3335543984388589
0.10997026901743896
0.08530214620219882
0.03977193766130632
0.0067117894509761645
0.03137699771689497
0.0037383524520176706
0.00280574458683164
0.48381070983810615
0.45393835616438355
0.04452054794520548
0.010958904109589041
0.7883561643835616
0.14931506849315068
Pave    0.995890
Pave    0.028082
dtype: float64
Pave    0.995890
Pave    0.028082
dtype: float64
0.028082191780821917
0.00684931506849315
0.6335616438356164
0.03424657534246575
0.024657534246575342

In [2]:
pip install pandas-profiling 

Note: you may need to restart the kernel to use updated packages.
